### WGAN与原始版本GAN和DCGAN差异较大，请仔细阅读文档并补充完整下面的代码。在需要补充的部分已经标注#TODO并附上相应的内容提示。

In [2]:
# 导入必要的库
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os
from torch.utils.tensorboard import SummaryWriter  # TensorBoard

#### 根据文档和提示，补充完整WGAN的生成器Generator和判别器Discriminator代码：
注意：
1. WGAN 生成器的设计与 DCGAN 类似，但会使用 Tanh 激活函数将图像的像素值限制在 [-1, 1] 的范围内。
2. 在 WGAN 中，判别器输出的是一个实数，而不是概率，因此不使用 sigmoid 激活函数。

In [3]:
# =============================== 生成器（Generator） ===============================
class Generator(nn.Module):
    def __init__(self, input_dim):
        super(Generator, self).__init__()

        # 1. 输入层：将 100 维随机噪声从input_dim投影到 32x32（1024 维）
        #TODO   # 线性变换fc1，将输入噪声扩展到 1024 维
        self.fc1 = nn.Linear(100,32*32)

        self.br1 = nn.Sequential(
            #TODO   # 批归一化，加速训练并稳定收敛
            nn.BatchNorm1d(32*32),
            #TODO   # ReLU 激活函数，引入非线性
            nn.ReLU()
        )

        # 2. 第二层：将 1024 维数据映射到 128 * 7 * 7 维
        #TODO   # 线性变换，将数据变换为适合卷积层的维数大小
        self.fc2 = nn.Linear(1024,128*7*7)
        self.br2 = nn.Sequential(
            #TODO   # 批归一化
            nn.BatchNorm1d(128*7*7),
            #TODO   # ReLU 激活函数
            nn.ReLU()
        )

        # 3. 反卷积层 1：上采样，输出 64 通道的 14×14 特征图
        self.conv1 = nn.Sequential(
            #TODO   # 反卷积：将 7x7 放大到 14x14，kernel size设置为4，stride设置为2，padding设置为1
            nn.ConvTranspose2d(in_channels=128 ,out_channels=64,kernel_size=4, stride=2, padding=1),
            #TODO   # 归一化，稳定训练
            nn.BatchNorm2d(64),
            #TODO   # ReLU 激活函数
            nn.ReLU()
        )

        # 4. 反卷积层 2：输出 1 通道的 28×28 图像
        self.conv2 = nn.Sequential(
            #TODO   # 反卷积：将 14x14 放大到 28x28，将 7x7 放大到 14x14，kernel size设置为4，stride设置为2，padding设置为1
            nn.ConvTranspose2d(in_channels=64 ,out_channels=1 ,kernel_size=4, stride=2, padding=1),
            #TODO    # WGAN 需要使用 Tanh 激活函数，将输出范围限制在 [-1, 1]
            nn.Tanh()
        )

    def forward(self, x):
        x = self.br1(self.fc1(x))  # 通过全连接层，进行 BatchNorm 和 ReLU 激活
        x = self.br2(self.fc2(x))  # 继续通过全连接层，进行 BatchNorm 和 ReLU 激活
        x = x.reshape(-1, 128, 7, 7)  # 变形为适合卷积输入的形状 (batch, 128, 7, 7)
        x = self.conv1(x)  # 反卷积：上采样到 14x14
        output = self.conv2(x)  # 反卷积：上采样到 28x28
        return output  # 返回生成的图像

# =============================== 判别器（Discriminator） ===============================
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        # 1. 第一层：输入 1 通道的 28x28 图像，输出 32 通道的特征图，然后通过MaxPool2d降采样
        self.conv1 = nn.Sequential(
             #TODO  # 5x5 卷积核，步长为1
            nn.Conv2d(in_channels=1, out_channels=32 ,kernel_size=5, stride=1),
            #TODO   # LeakyReLU，negative_slope参数设置为0.1
            nn.LeakyReLU(negative_slope=0.1)
        )
        self.pl1 = nn.MaxPool2d(2, stride=2)

        # 2. 第二层：输入 32 通道，输出 64 通道特征, 然后通过MaxPool2d降采样
        self.conv2 = nn.Sequential(
            #TODO   # 5x5 卷积核，步长为1
            nn.Conv2d(in_channels=32, out_channels=64 ,kernel_size=5, stride=1),
            #TODO  # LeakyReLU 激活函数，negative_slope参数设置为0.1
            nn.LeakyReLU(negative_slope=0.1)
        )
        self.pl2 = nn.MaxPool2d(2, stride=2)

        # 3. 全连接层 1：将 64x4x4 维特征图转换成 1024 维向量
        self.fc1 = nn.Sequential(
           #TODO   # 线性变换，将 64x4x4 映射到 1024 维
            nn.Linear(64*4*4, 1024),
            #TODO   # LeakyReLU 激活函数，negative_slope参数设置为0.1
            nn.LeakyReLU(negative_slope=0.1)
        )

        # 4. 全连接层 2：最终输出
        #TODO # 输出一个标量作为判别结果
        self.fc2 = nn.Linear(1024,1)

    def forward(self, x):
        x = self.pl1(self.conv1(x))  # 第一层卷积，降维
        x = self.pl2(self.conv2(x))  # 第二层卷积，降维
        x = x.view(x.shape[0], -1)  # 展平成向量
        x = self.fc1(x)  # 通过全连接层
        output = self.fc2(x)  # 通过最后一层全连接层，输出标量
        return output  # 返回判别结果

#### 补充完整主函数。
注意：
1. 传统的GAN通常使用[0, 1]范围的图像作为输入，但WGAN要求图像的像素值在 [-1, 1] 范围内。此时，输入图像的像素值需要做归一化，使用 (0.5,) 作为均值 (0.5) 和 (0.5,) 作为标准差，确保每个像素的值都被调整到 [-1, 1] 之间。
2. 与传统的GAN使用Adam优化器不同，WGAN推荐使用RMSprop优化器。
3. 在WGAN中，通常需要在每次生成器训练之前，先训练判别器多次。这种策略有助于使判别器的训练更加稳定。

In [4]:
# =============================== 主函数 ===============================
def main():
    # 设备配置
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # 设定超参数
    input_dim = 100
    batch_size = 128
    num_epoch = 10
    clip_value = 0.01   # 判别器权重裁剪范围，确保满足 Lipschitz 条件

    # 加载 MNIST 数据集
    train_dataset = datasets.MNIST(root="./data/", train=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]), download=True)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    # 创建生成器G和判别器D，并移动到 GPU（如果可用）
    ## TODO
    G = Generator(input_dim).to(device)
    # TODO
    D = Discriminator().to(device)
    
    # 定义优化器optim_G和optim_D：使用RMSprop，学习率设置为0.00005
    # TODO
    optim_G = torch.optim.RMSprop(G.parameters(), lr = 0.00005)
    # TODO
    optim_D = torch.optim.RMSprop(D.parameters(), lr = 0.00005)
    # 初始化 TensorBoard
    writer = SummaryWriter(log_dir='./logs/experiment_wgan')

    # 开始训练
    for epoch in range(num_epoch):
        total_loss_D, total_loss_G = 0, 0
        for i, (real_images, _) in enumerate(train_loader):
            # 判别器训练5次，然后训练生成器1次。提示：for循环，记得修改total_loss_D和total_loss_G的值
            # TODO  # 判别器训练 5 次
            for __ in range(5):
                loss_D = train_discriminator(real_images, D, G, optim_D, clip_value, batch_size, input_dim, device)
                total_loss_D += loss_D
            # TODO  # 生成器训练 1 次
            loss_G = train_generator(D, G, optim_G, batch_size, input_dim, device)
            total_loss_G += loss_G

            # 每 100 步打印一次损失
            if (i + 1) % 100 == 0 or (i + 1) == len(train_loader):
                print(f'Epoch {epoch:02d} | Step {i + 1:04d} / {len(train_loader)} | Loss_D {total_loss_D / (i + 1):.4f} | Loss_G {total_loss_G / (i + 1):.4f}')

        # 记录损失到 TensorBoard
        writer.add_scalar('WGAN/Loss/Discriminator', total_loss_D / len(train_loader), epoch)
        writer.add_scalar('WGAN/Loss/Generator', total_loss_G / len(train_loader), epoch)

        # 生成并保存示例图像
        with torch.no_grad():
            noise = torch.randn(64, input_dim, device=device)
            fake_images = G(noise)

            # 记录生成的图像到 TensorBoard
            img_grid = torchvision.utils.make_grid(fake_images, normalize=True)
            writer.add_image('Generated Images', img_grid, epoch)

    writer.close()

#### 根据文档中描述的GAN的损失函数和二元交叉熵损失相关内容，补充完善Discriminator和Generator的训练过程：
注意：
1. 判别器需要进行权重裁剪操作；
2. 生成器和判别器的损失函数与GAN和DCGAN不同。

In [5]:
# =============================== 训练判别器 ===============================
def train_discriminator(real_images, D, G, optim_D, clip_value, batch_size, input_dim, device):
    '''训练判别器'''
    
    real_images = real_images.to(device)
    real_output = D(real_images)
    
    noise = torch.randn(batch_size, input_dim, device=device)
    fake_images = G(noise).detach()
    fake_output = D(fake_images)

    #TODO  # 计算判别器的损失函数
    loss_D = -(torch.mean(real_output) - torch.mean(fake_output))
    optim_D.zero_grad()
    loss_D.backward()
    optim_D.step()

    # 对判别器参数进行裁剪
    for p in D.parameters():
        p.data.clamp_(-clip_value, clip_value)

    return loss_D.item()

# =============================== 训练生成器 ===============================
def train_generator(D, G, optim_G, batch_size, input_dim, device):
    '''训练生成器'''
    
    noise = torch.randn(batch_size, input_dim, device=device)
    fake_images = G(noise)
    fake_output = D(fake_images)

    #TODO  # 计算生成器的损失函数
    loss_G = -torch.mean(fake_output)
    optim_G.zero_grad()
    loss_G.backward()
    optim_G.step()

    return loss_G.item()

In [6]:
if __name__ == '__main__':
    main()

Epoch 00 | Step 0100 / 469 | Loss_D -17.3652 | Loss_G 1.6715
Epoch 00 | Step 0200 / 469 | Loss_D -18.8381 | Loss_G -2.9450
Epoch 00 | Step 0300 / 469 | Loss_D -13.9938 | Loss_G -4.9737
Epoch 00 | Step 0400 / 469 | Loss_D -11.6035 | Loss_G -5.3218
Epoch 00 | Step 0469 / 469 | Loss_D -10.3998 | Loss_G -5.3675
Epoch 01 | Step 0100 / 469 | Loss_D -3.1365 | Loss_G -4.4688
Epoch 01 | Step 0200 / 469 | Loss_D -2.8635 | Loss_G -4.1194
Epoch 01 | Step 0300 / 469 | Loss_D -2.6523 | Loss_G -3.7522
Epoch 01 | Step 0400 / 469 | Loss_D -2.4832 | Loss_G -3.4512
Epoch 01 | Step 0469 / 469 | Loss_D -2.3856 | Loss_G -3.2829
Epoch 02 | Step 0100 / 469 | Loss_D -1.7180 | Loss_G -1.6266
Epoch 02 | Step 0200 / 469 | Loss_D -1.6418 | Loss_G -1.5405
Epoch 02 | Step 0300 / 469 | Loss_D -1.5775 | Loss_G -1.5430
Epoch 02 | Step 0400 / 469 | Loss_D -1.5056 | Loss_G -1.5182
Epoch 02 | Step 0469 / 469 | Loss_D -1.4612 | Loss_G -1.4824
Epoch 03 | Step 0100 / 469 | Loss_D -1.1411 | Loss_G -1.4182
Epoch 03 | Step 0200